In [29]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter
import numpy as np
import itertools
import copy
import matplotlib.pyplot as plt
import json
import time
from dataraw_sampling import (sample_queries, 
                                         sample_one_training_datapoint, 
                                         construct_cardpair_answer_lookup, 
                                         sample_queries, resolve_eval_expression, 
                                         decode_key_to_vocab_token)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## SET Shatter

In [23]:
# Simple Shatter

num_attributes=4; num_attr_vals=3; query_length_multiplier=1; nest_depth_int=1; multiple_OR_sets_bool=True
N_train=0; N_val=5120; N_test=5120

expression_resolve_fn = resolve_eval_expression

sample_start_time = time.time()
data, stats = sample_queries(
    num_attributes, query_length_multiplier, nest_depth_int, multiple_OR_sets_bool, 
    expression_resolve_fn,
    N_train, N_val, N_test,
    validate=True,
    debug=False)
sample_end_time = time.time()

print(f'Sample Time for {N_train+N_val+N_test} datapoints = {sample_end_time-sample_start_time} seconds')


save_start_time = time.time()
filename1 = '../Raw_Datasets/SET/SETShatter-{}Attr-{}Vals-{}LenMultiplier-{}NestDepth-{}Train-{}Val-{}Test.json'.format(
    num_attributes, num_attr_vals, query_length_multiplier, nest_depth_int, N_train, N_val, N_test)
with open(filename1, 'w') as f:
    json.dump(data, f)
save_end_time = time.time()

filename = '../Raw_Datasets/SET/SETShatter-{}Attr-{}Vals-{}LenMultiplier-{}NestDepth-{}Train-{}Val-{}Test-Stats.json'.format(
    num_attributes, num_attr_vals, query_length_multiplier, nest_depth_int, N_train, N_val, N_test)
with open(filename, 'w') as f:
    json.dump(stats, f)
save_end_time = time.time()

print(f'Saving Time for {N_train+N_val+N_test} datapoints = {save_end_time-save_start_time} seconds')

100%|██████████| 10240/10240 [01:15<00:00, 134.80it/s]


Sample Time for 10240 datapoints = 76.05536222457886 seconds
Saving Time for 10240 datapoints = 1.156684160232544 seconds


In [28]:
data['val_tokens'][0][1]

[54]

In [35]:
filename1

'../Raw_Datasets/SET/SETShatter-4Attr-3Vals-1LenMultiplier-1NestDepth-0Train-5120Val-5120Test.json'

In [8]:
filename1

'../Raw_Datasets/SET/SETShatter-3Attr-3Vals-1LenMultiplier-1NestDepth-0Train-5120Val-5120Test.json'

## SET Shatter - one datapoint only

In [30]:
num_attributes=3; num_attr_vals=3; query_length_multiplier=1; nest_depth_int=1; multiple_OR_sets_bool=True
N_train=0; N_val=5120; N_test=5120

cardpair_answer_lookup = construct_cardpair_answer_lookup(num_attributes, num_attr_vals)
base_vocab_size =  num_attr_vals ** num_attributes
symbol_vocab_token_lookup = {
    '(': base_vocab_size,
    ')': base_vocab_size + 1,
    'NULL': base_vocab_size + 2,
    'SEP': base_vocab_size + 3,
    'SOS': base_vocab_size + 4,
    'EOS': base_vocab_size + 5,
    'PAD': base_vocab_size + 6,
    'PLH': base_vocab_size + 7,
    '&': base_vocab_size + 8,
    '|': base_vocab_size + 9,
}


q_tokens, k_idx, gt_idxs = sample_one_training_datapoint(
    2, 
    num_attributes, 
    num_attr_vals, 
    query_length_multiplier, 
    nest_depth_int, 
    multiple_OR_sets_bool,
    cardpair_answer_lookup, 
    symbol_vocab_token_lookup,
    validate=True, debug=True)


gt_keys_idx [9, 18]

gt key properties [array([1, 0, 0]), array([2, 0, 0])]

flat_card_indices [['OR', 24, 17], 22, ['OR', 21, 25], ['OR', 13, 12], ['OR', 5, 10], 26, ['OR', 9, 13], 18, ['OR', 2, 25], 4, ['OR', 14, 19], 6, ['OR', 15, 1], ['OR', 16, 18], 'DERIVE', 6, 9]

query_eval_expression ['AND', ['OR', 24, 17], 22, ['OR', 21, 25], ['OR', 13, 12], ['OR', 5, 10], 26, ['OR', 9, 13], 18, ['OR', 2, 25], 4, ['OR', 14, 19], 6, ['OR', 15, 1], ['OR', 16, 18], 'DERIVE', 6, 9]

query_input_expression ['(', '(', 24, '|', 17, ')', '&', 22, '&', '(', 21, '|', 25, ')', '&', '(', 13, '|', 12, ')', '&', '(', 5, '|', 10, ')', '&', 26, '&', '(', 9, '|', 13, ')', '&', 18, '&', '(', 2, '|', 25, ')', '&', 4, '&', '(', 14, '|', 19, ')', '&', 6, '&', '(', 15, '|', 1, ')', '&', '(', 16, '|', 18, ')', '&', 'DERIVE', '&', 6, '&', 9, ')']
lhs_expression ['AND', ['OR', 24, 17], 22, ['OR', 21, 25], ['OR', 13, 12], ['OR', 5, 10], 26, ['OR', 9, 13], 18, ['OR', 2, 25], 4, ['OR', 14, 19], 6, ['OR', 15, 1], ['OR', 

In [34]:
len(q_tokens)

75

In [33]:
gt_idxs

[9, 18]

In [31]:
k_idx

[18]

In [32]:
q_tokens

[27,
 27,
 24,
 36,
 17,
 28,
 35,
 22,
 35,
 27,
 21,
 36,
 25,
 28,
 35,
 27,
 13,
 36,
 12,
 28,
 35,
 27,
 5,
 36,
 10,
 28,
 35,
 26,
 35,
 27,
 9,
 36,
 13,
 28,
 35,
 18,
 35,
 27,
 2,
 36,
 25,
 28,
 35,
 4,
 35,
 27,
 14,
 36,
 19,
 28,
 35,
 6,
 35,
 27,
 15,
 36,
 1,
 28,
 35,
 27,
 16,
 36,
 18,
 28,
 35,
 27,
 6,
 36,
 18,
 28,
 35,
 6,
 35,
 9,
 28]